In [1]:
# %cd /content/drive/Shared drives/IML Project/Project

In [18]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [19]:
import tensorflow as tf

from tensorflow.keras import Sequential

from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Conv3D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import MaxPool3D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import AveragePooling3D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten

In [20]:
tf.config.list_physical_devices('GPU')

[]

In [21]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  0


In [22]:
data_path = 'data'
labels = pd.read_csv(os.path.join(data_path, 'adni_demographic_master_kaggle.csv'))

In [23]:
def load_datasets(type: str):
    if type not in ['train', 'test', 'valid']: raise Exception('Unsupported dataset type')
    train_valid_test = 0 if type == 'train' else 1 if type == 'valid' else 2
    i = 1
    dataset = np.load(os.path.join(data_path, f'img_array_{type}_6k_{i}.npy'))
    while True:
        try:
            i += 1
            dataset = np.vstack((dataset, np.load(os.path.join(data_path, f'img_array_{type}_6k_{i}.npy'))))
        except FileNotFoundError:
            print(f'Loaded all {type} datasets')
            break
    # dataset = np.expand_dims(dataset, axis=1)
    dataset = np.reshape(dataset, (-1, 62, 96, 96, 1))
    for i in range(dataset.shape[0]):
        dataset[i] = dataset[i]/np.amax(dataset[i])
    return dataset, np.eye(3)[labels[labels.train_valid_test == train_valid_test].diagnosis - 1]

In [24]:
test_data, test_labels = load_datasets('test')
train_data, train_labels = load_datasets('train')
valid_data, valid_labels = load_datasets('valid')

Loaded all test datasets


MemoryError: Unable to allocate 4.94 GiB for an array with shape (72000, 96, 96) and data type float64

In [16]:
train_data.shape

(2109, 62, 96, 96, 1)

In [ ]:
# train_data2 = tf.data.Dataset.from_tensor_slices((train_data, train_labels))
# train_data2

In [ ]:
def create_model():
    kernel_regularizer = tf.keras.regularizers.l2(0.001)
    he_normal = tf.keras.initializers.he_normal(seed=0)
    lecun_normal = tf.keras.initializers.lecun_normal(seed=0)

    input_3d = (62, 96, 96, 1)
    pool_3d = (2, 2, 2)

    model = Sequential()
    
    model.add(tf.keras.layers.InputLayer(input_shape=input_3d))
    model.add(Conv3D(filters=8,
                     kernel_size=3,
                     kernel_regularizer=kernel_regularizer,
                     kernel_initializer=he_normal))
    model.add(BatchNormalization())
    model.add(LeakyReLU())
    model.add(MaxPool3D(pool_size=pool_3d, name='pool1'))

    model.add(Conv3D(filters=8,
                     kernel_size=3,
                     kernel_regularizer=kernel_regularizer,
                     kernel_initializer=he_normal))
    model.add(BatchNormalization())
    model.add(LeakyReLU())
    model.add(MaxPool3D(pool_size=pool_3d, name='pool2'))

    model.add(Conv3D(filters=8,
                     kernel_size=3,
                     kernel_regularizer=kernel_regularizer,
                     kernel_initializer=he_normal))
    model.add(BatchNormalization())
    model.add(LeakyReLU())
    model.add(MaxPool3D(pool_size=pool_3d, name='pool3'))

    model.add(Flatten())
    model.add(Dense(1024, name='dense1', kernel_regularizer=kernel_regularizer, kernel_initializer=he_normal))
    model.add(LeakyReLU())
    model.add(Dropout(0.5, name='dropout1'))

    model.add(Dense(512, activation='relu', name='dense2', kernel_regularizer=kernel_regularizer, kernel_initializer=he_normal))
    model.add(LeakyReLU())
    model.add(Dropout(0.5, name='dropout2'))

    model.add(Dense(3, activation='softmax', name='softmax', kernel_initializer=lecun_normal))

    model.compile(loss='categorical_crossentropy',
                  optimizer=tf.keras.optimizers.Adam(),
                  metrics=[tf.keras.metrics.CategoricalAccuracy()])

    return model

In [ ]:
reducelr_callback = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_categorical_accuracy',
                                                         factor=0.2,
                                                         patience=5,
                                                         min_delta=0.01,
                                                         verbose=1)
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(os.path.join('weights', 'best_weights.h5'),
                                                         monitor='val_categorical_accuracy',
                                                         verbose=1,
                                                         save_best_only=True,
                                                         mode='max')

callbacks_list = [checkpoint_callback, reducelr_callback]

In [ ]:
model = create_model()
model.summary()

In [16]:
# gpu_options = tf.GPUOptions(allow_growth=True)
# session = tf.InteractiveSession(config=tf.ConfigProto(gpu_options=gpu_options))

In [ ]:
history = model.fit(x=train_data, y=train_labels, batch_size=32, validation_data=(valid_data, valid_labels), epochs=25, callbacks=callbacks_list, verbose=1)